# Building the dataset of research papers

The [Entrez](http://biopython.org/DIST/docs/api/Bio.Entrez-module.html) module, a part of the [Biopython](http://biopython.org/) library, will be used to interface with [PubMed](http://www.ncbi.nlm.nih.gov/pubmed).<br>
You can download Biopython from [here](http://biopython.org/wiki/Download).

In this notebook we will be covering several of the steps taken in the [Biopython Tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html), specifically in [Chapter 9  Accessing NCBI’s Entrez databases](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc109).

In [1]:
from Bio import Entrez

# NCBI requires you to set your email address to make use of NCBI's E-utilities
Entrez.email = "Your.Name.Here@example.org"

The datasets will be saved as serialized Python objects, compressed with bzip2.
Saving/loading them will therefore require the [pickle](http://docs.python.org/3/library/pickle.html) and [bz2](http://docs.python.org/3/library/bz2.html) modules.

In [2]:
import pickle, bz2, os

## EInfo: Obtaining information about the Entrez databases

In [3]:
# accessing extended information about the PubMed database
pubmed = Entrez.read( Entrez.einfo(db="pubmed"), validate=False )[u'DbInfo']

# list of possible search fields for use with ESearch:
search_fields = { f['Name']:f['Description'] for f in pubmed["FieldList"] }

In search_fields, we find 'TIAB' ('Free text associated with Abstract/Title') as a possible search field to use in searches.

In [4]:
search_fields

{'AFFL': "Author's institutional affiliation and address",
 'ALL': 'All terms from all searchable fields',
 'AUCL': 'Author Cluster ID',
 'AUID': 'Author Identifier',
 'AUTH': 'Author(s) of publication',
 'BOOK': 'ID of the book that contains the document',
 'CDAT': 'Date of completion',
 'CNTY': 'Country of publication',
 'COIS': 'Conflict of Interest Statements',
 'COLN': 'Corporate Author of publication',
 'CRDT': 'Date publication first accessible through Entrez',
 'DSO': 'Additional text from the summary',
 'ECNO': 'EC number for enzyme or CAS registry number',
 'ED': "Section's Editor",
 'EDAT': 'Date publication first accessible through Entrez',
 'EID': 'Extended PMID',
 'EPDT': 'Date of Electronic publication',
 'FAUT': 'First Author of publication',
 'FILT': 'Limits the records',
 'FINV': 'Full name of investigator',
 'FULL': 'Full Author Name(s) of publication',
 'GRNT': 'NIH Grant Numbers',
 'INVR': 'Investigator',
 'ISBN': 'ISBN',
 'ISS': 'Issue number of publication',
 'JO

## ESearch: Searching the Entrez databases

To have a look at the kind of data we get when searching the database, we'll perform a search for papers authored by Haasdijk:

In [5]:
example_authors = ['Haasdijk E']
example_search = Entrez.read( Entrez.esearch( db="pubmed", term=' AND '.join([a+'[AUTH]' for a in example_authors]) ) )
example_search

{'Count': '36', 'RetMax': '20', 'RetStart': '0', 'IdList': ['29311830', '28513205', '28513201', '28323435', '28140628', '26933487', '24977986', '24901702', '24852945', '24708899', '24252306', '23580075', '23144668', '22174697', '22154920', '21870131', '21760539', '20662596', '20602234', '20386726'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'Haasdijk E[Author]', 'Field': 'Author', 'Count': '36', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'Haasdijk E[Author]'}

Note how the result being produced is not in Python's native string format:

In [6]:
type( example_search['IdList'][0] )

Bio.Entrez.Parser.StringElement

The part of the query's result we are most interested in is accessible through

In [7]:
example_ids = [ int(id) for id in example_search['IdList'] ]
print(example_ids)

[29311830, 28513205, 28513201, 28323435, 28140628, 26933487, 24977986, 24901702, 24852945, 24708899, 24252306, 23580075, 23144668, 22174697, 22154920, 21870131, 21760539, 20662596, 20602234, 20386726]


### PubMed IDs dataset

We will now assemble a dataset comprised of research articles containing the given keyword, in either their titles or abstracts.

In [8]:
search_term = 'influenza'

In [9]:
Ids_file = 'data/' + search_term + '_Ids.pkl.bz2'

In [10]:
if os.path.exists( Ids_file ):
    Ids = pickle.load( bz2.BZ2File( Ids_file, 'rb' ) )
else:
    # determine the number of hits for the search term
    search = Entrez.read( Entrez.esearch( db="pubmed", term=search_term+'[TIAB]', retmax=0 ) )
    total = int( search['Count'] )
    
    # `Ids` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Ids_str = []
    retrieve_per_query = 10000
    
    for start in range( 0, total, retrieve_per_query ):
        print('Fetching IDs of results [%d,%d]' % ( start, start+retrieve_per_query ) )
        s = Entrez.read( Entrez.esearch( db="pubmed", term=search_term+'[TIAB]', retstart=start, retmax=retrieve_per_query ) )
        Ids_str.extend( s[ u'IdList' ] )
    
    # convert Ids to integers (and ensure that the conversion is reversible)
    Ids = [ int(id) for id in Ids_str ]
    
    for (id_str, id_int) in zip(Ids_str, Ids):
        if str(id_int) != id_str:
            raise Exception('Conversion of PubMed ID %s from string to integer it not reversible.' % id_str )
    
    # Remove IDs that would cause problems below:
    Ids.remove(28078898)
    Ids.remove(7467326)
    Ids.remove(27319233)
    Ids.remove(26084645)
    
    # Save list of Ids
    pickle.dump( Ids, bz2.BZ2File( Ids_file, 'wb' ) )
    
total = len( Ids )
print('%d documents contain the search term "%s".' % ( total, search_term ) )

87693 documents contain the search term "influenza".


Taking a look at what we just retrieved, here are the last 5 elements of the `Ids` list:

In [11]:
Ids[:5]

[30248524, 30248474, 30248374, 30248373, 30248106]

## ESummary: Retrieving summaries from primary IDs

To have a look at the kind of metadata we get from a call to `Entrez.esummary()`, we now fetch the summary of one of Haasdijk's papers (using one of the PubMed IDs we obtained in the previous section:

In [12]:
example_paper = Entrez.read( Entrez.esummary(db="pubmed", id='30227210') )[0]

def print_dict( p ):
    for k,v in p.items():
        print(k)
        print('\t', v)

print_dict(example_paper)

Item
	 []
Id
	 30227210
PubDate
	 2018 Sep 15
EPubDate
	 2018 Sep 15
Source
	 Int J Biol Macromol
AuthorList
	 ['Li Q', 'Wang Q', 'Jin X', 'Chen Z', 'Xiong C', 'Li P', 'Zhao J', 'Huang W']
LastAuthor
	 Huang W
Title
	 The first complete mitochondrial genome from the family Hygrophoraceae (Hygrophorus russula) by next-generation sequencing and phylogenetic implications.
Volume
	 
Issue
	 
Pages
	 
LangList
	 ['English']
NlmUniqueID
	 7909578
ISSN
	 0141-8130
ESSN
	 1879-0003
PubTypeList
	 ['Journal Article']
RecordStatus
	 PubMed - as supplied by publisher
PubStatus
	 aheadofprint
ArticleIds
	 {'pubmed': ['30227210'], 'medline': [], 'pii': 'S0141-8130(18)34352-6', 'doi': '10.1016/j.ijbiomac.2018.09.091', 'rid': '30227210', 'eid': '30227210'}
DOI
	 10.1016/j.ijbiomac.2018.09.091
History
	 {'pubmed': ['2018/09/19 06:00'], 'medline': ['2018/09/19 06:00'], 'received': '2018/08/18 00:00', 'revised': '2018/09/13 00:00', 'accepted': '2018/09/14 00:00', 'entrez': '2018/09/19 06:00'}
References


For now, we'll keep just some basic information for each paper: title, list of authors, publication year, and [DOI](https://en.wikipedia.org/wiki/Digital_object_identifier).

In case you are not familiar with the DOI system, know that the paper above can be accessed through the link  `https://www.doi.org/` followed by the paper's DOI.

In [13]:
( example_paper['Title'], example_paper['AuthorList'], int(example_paper['PubDate'][:4]), example_paper['DOI'] )

('The first complete mitochondrial genome from the family Hygrophoraceae (Hygrophorus russula) by next-generation sequencing and phylogenetic implications.',
 ['Li Q', 'Wang Q', 'Jin X', 'Chen Z', 'Xiong C', 'Li P', 'Zhao J', 'Huang W'],
 2018,
 '10.1016/j.ijbiomac.2018.09.091')

### Summaries dataset

We are now ready to assemble a dataset containing the summaries of all the paper `Ids` we previously fetched.

To reduce the memory footprint, and to ensure the saved datasets won't depend on Biopython being installed to be properly loaded, values returned by `Entrez.read()` will be converted to their corresponding native Python types. We start by defining a function for helping with the conversion of strings:

In [14]:
Summaries_file = 'data/' + search_term + '_Summaries.pkl.bz2'

In [15]:
if os.path.exists( Summaries_file ):
    Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
else:
    # `Summaries` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Summaries = []
    retrieve_per_query = 500
    
    print('Fetching Summaries of results: ')
    for start in range( 0, len(Ids), retrieve_per_query ):
        if (start % 10000 == 0):
            print('')
            print(start, end='')
        else:
            print('.', end='')
        
        # build comma separated string with the ids at indexes [start, start+retrieve_per_query)
        query_ids = ','.join( [ str(id) for id in Ids[ start : start+retrieve_per_query ] ] )
        
        s = Entrez.read( Entrez.esummary( db="pubmed", id=query_ids ) )
        
        # out of the retrieved data, we will keep only a tuple (title, authors, year, DOI), associated with the paper's id.
        # (all values converted to native Python formats)
        for p in s:
            try:
                f = [
                    ( int( p['Id'] ), (
                        str( p['Title'] ),
                        [ str(a) for a in p['AuthorList'] ],
                        int( p['PubDate'][:4] ),                # keeps just the publication year
                        str( p.get('DOI', '') )            # papers for which no DOI is available get an empty string in their place
                        ) )
                    ]
                Summaries.extend( f )
            except ValueError as e:
                print("\nError with ID " + p['Id'] + ": " + str(e))
                print("Manually remove this ID above and re-run code.")
    
    # Save Summaries, as a dictionary indexed by Ids
    Summaries = dict( Summaries )
    
    pickle.dump( Summaries, bz2.BZ2File( Summaries_file, 'wb' ) )

Let us take a look at the first 3 retrieved summaries:

In [16]:
{ id : Summaries[id] for id in Ids[:3] }

{30248374: ('Influenza virus replication raises the temperature of cells.',
  ['Maruyama H',
   'Kimura T',
   'Liu H',
   'Ohtsuki S',
   'Miyake Y',
   'Isogai M',
   'Arai F',
   'Honda A'],
  2018,
  '10.1016/j.virusres.2018.09.011'),
 30248474: ('A family cluster of two fatal cases infected with influenza A (H7N9) virus in Kunming China, 2017.',
  ['Guo L',
   'Hou M',
   'Ning R',
   'Li W',
   'Yang Z',
   'Li H',
   'Chu M',
   'Yu L',
   'Liu L'],
  2018,
  '10.1016/j.meegid.2018.09.022'),
 30248524: ('Combined use of live-attenuated and inactivated influenza vaccines to enhance heterosubtypic protection.',
  ['Yan LM',
   'Li OTW',
   'Poh CM',
   'Perera RAPM',
   'Valkenburg SA',
   'Peiris M',
   'Poon LLM'],
  2018,
  '10.1016/j.virol.2018.09.007')}

## EFetch: Downloading full records from Entrez

`Entrez.efetch()` is the function that will allow us to obtain paper abstracts. Let us start by taking a look at the kind of data it returns when we query PubMed's database.

In [17]:
q = Entrez.read( Entrez.efetch(db="pubmed", id='30227210', retmode="xml") )['PubmedArticle']

`q` is a list, with each member corresponding to a queried id. Because here we only queried for one id, its results are then in `q[0]`.

In [18]:
type(q), len(q)

(list, 1)

1
At `q[0]` we find a dictionary containing two keys, the contents of which we print below.

In [19]:
type(q[0]), q[0].keys()

(Bio.Entrez.Parser.DictionaryElement,
 dict_keys(['MedlineCitation', 'PubmedData']))

In [20]:
print_dict( q[0][ 'PubmedData' ] )

History
	 [DictElement({'Year': '2018', 'Month': '08', 'Day': '18'}, attributes={'PubStatus': 'received'}), DictElement({'Year': '2018', 'Month': '09', 'Day': '13'}, attributes={'PubStatus': 'revised'}), DictElement({'Year': '2018', 'Month': '09', 'Day': '14'}, attributes={'PubStatus': 'accepted'}), DictElement({'Year': '2018', 'Month': '9', 'Day': '19', 'Hour': '6', 'Minute': '0'}, attributes={'PubStatus': 'pubmed'}), DictElement({'Year': '2018', 'Month': '9', 'Day': '19', 'Hour': '6', 'Minute': '0'}, attributes={'PubStatus': 'medline'}), DictElement({'Year': '2018', 'Month': '9', 'Day': '19', 'Hour': '6', 'Minute': '0'}, attributes={'PubStatus': 'entrez'})]
PublicationStatus
	 aheadofprint
ArticleIdList
	 [StringElement('30227210', attributes={'IdType': 'pubmed'}), StringElement('S0141-8130(18)34352-6', attributes={'IdType': 'pii'}), StringElement('10.1016/j.ijbiomac.2018.09.091', attributes={'IdType': 'doi'})]


The key `'MedlineCitation'` maps into another dictionary. In that dictionary, most of the information is contained under the key `'Article'`. To minimize the clutter, below we show the contents of `'MedlineCitation'` excluding its `'Article'` member, and below that we then show the contents of `'Article'`.

In [21]:
print_dict( { k:v for k,v in q[0][ 'MedlineCitation' ].items() if k!='Article' } )

CitationSubset
	 []
OtherID
	 []
OtherAbstract
	 []
KeywordList
	 [ListElement([StringElement('Gene rearrangement', attributes={'MajorTopicYN': 'N'}), StringElement('Hygrophorus', attributes={'MajorTopicYN': 'N'}), StringElement('Mitochondrial genome', attributes={'MajorTopicYN': 'N'}), StringElement('Phylogenetic analysis', attributes={'MajorTopicYN': 'N'}), StringElement('Protein-encoding gene', attributes={'MajorTopicYN': 'N'}), StringElement('Repeat sequence', attributes={'MajorTopicYN': 'N'})], attributes={'Owner': 'NOTNLM'})]
SpaceFlightMission
	 []
GeneralNote
	 []
PMID
	 30227210
DateRevised
	 {'Year': '2018', 'Month': '09', 'Day': '21'}
MedlineJournalInfo
	 {'Country': 'Netherlands', 'MedlineTA': 'Int J Biol Macromol', 'NlmUniqueID': '7909578', 'ISSNLinking': '0141-8130'}


In [22]:
print_dict( q[0][ 'MedlineCitation' ][ 'Article' ] )

ELocationID
	 [StringElement('S0141-8130(18)34352-6', attributes={'EIdType': 'pii', 'ValidYN': 'Y'}), StringElement('10.1016/j.ijbiomac.2018.09.091', attributes={'EIdType': 'doi', 'ValidYN': 'Y'})]
Language
	 ['eng']
ArticleDate
	 [DictElement({'Year': '2018', 'Month': '09', 'Day': '15'}, attributes={'DateType': 'Electronic'})]
Journal
	 {'ISSN': StringElement('1879-0003', attributes={'IssnType': 'Electronic'}), 'JournalIssue': DictElement({'PubDate': {'Year': '2018', 'Month': 'Sep', 'Day': '15'}}, attributes={'CitedMedium': 'Internet'}), 'Title': 'International journal of biological macromolecules', 'ISOAbbreviation': 'Int. J. Biol. Macromol.'}
ArticleTitle
	 The first complete mitochondrial genome from the family Hygrophoraceae (Hygrophorus russula) by next-generation sequencing and phylogenetic implications.
Abstract
	 {'AbstractText': ['Hygrophorus russula (Schaeff.) Kauffman is an edible ectomycorrhizal fungus that is widely distributed in the world. In this study, the mitogenome 

A paper's abstract can therefore be accessed with:

In [23]:
{ int(q[0]['MedlineCitation']['PMID']) : str(q[0]['MedlineCitation']['Article']['Abstract']['AbstractText'][0]) }

{30227210: 'Hygrophorus russula (Schaeff.) Kauffman is an edible ectomycorrhizal fungus that is widely distributed in the world. In this study, the mitogenome of H. russula was sequenced and assembled. The mitogenome of H. russula is composed of circular DNA molecules, with a total size of 55,769\u202fbp. Further analysis indicated that the frequent use of A and T in codons contributes to the high AT content (80.87%) in the H. russula mitogenome. Comparative analysis indicated that the length and base composition of the core protein-encoding genes, and the number of tRNA genes in the H. russula mitogenome varied from that of other Agaricales mitogenomes. Gene arrangement analysis revealed a novel gene order in the H. russula mitogenome. In addition, the expansion of the mitogenome in Agaricales was found to be closely related to the increase in the number of introns. Phylogenetic analysis of the combined mitochondrial gene set showed strong support for tree topologies, and H. russula w

Some of the ids in our dataset refer to books from the [NCBI Bookshelf](http://www.ncbi.nlm.nih.gov/books/), a collection of freely available, downloadable, on-line versions of selected biomedical books. For such ids, `Entrez.efetch()` returns a slightly different structure, where the keys `[u'BookDocument', u'PubmedBookData']` take the place of the `[u'MedlineCitation', u'PubmedData']` keys we saw above.

### Abstracts dataset

We can now assemble a dataset mapping paper ids to their abstracts.

In [24]:
Abstracts_file = 'data/' + search_term + '_Abstracts.pkl.bz2'

In [25]:
import http.client
from collections import deque
from xml.dom import minidom
import re

def ch(node, childtype):
    return node.getElementsByTagName(childtype)[0]

if os.path.exists( Abstracts_file ):
    Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )
else:
    # `Abstracts` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Abstracts = deque()
    retrieve_per_query = 500
    
    print('Fetching Abstracts of results: ')
    for start in range( 0, len(Ids), retrieve_per_query ):
        if (start % 10000 == 0):
            print('')
            print(start, end='')
        else:
            print('.', end='')
        
        # build comma separated string with the ids at indexes [start, start+retrieve_per_query)
        query_ids = ','.join( [ str(id) for id in Ids[ start : start+retrieve_per_query ] ] )
        
        # issue requests to the server, until we get the full amount of data we expect
        while True:
            try:
                #s = Entrez.read( Entrez.efetch(db="pubmed", id=query_ids, retmode="xml" ) )['PubmedArticle']
                s = minidom.parse( Entrez.efetch(db="pubmed", id=query_ids, retmode="xml" ) ).getElementsByTagName("PubmedArticle")
            except http.client.IncompleteRead:
                print('r', end='')
                continue
            break
        
        i = 0
        for p in s:
            abstr = ''
            if (p.getElementsByTagName('MedlineCitation')):
                citNode = ch(p,'MedlineCitation')
                pmid = ch(citNode,'PMID').firstChild.data
                articleNode = ch(citNode,'Article')
                if (articleNode.getElementsByTagName('Abstract')):
                    try:
                        abstr = ch(ch(articleNode,'Abstract'),'AbstractText').firstChild.data
                    except AttributeError:
                        abstr = ch(ch(articleNode,'Abstract'),'AbstractText').toprettyxml("  ")
                        abstr = re.sub(r"\s+", " ", re.sub("<[^>]*>", "", abstr))
            elif (p.getElementsByTagName('BookDocument')):
                bookNode = ch(p,'BookDocument')
                pmid = ch(bookNode,'PMID').firstChild.data
                if (bookNode.getElementsByTagName('Abstract')):
                    try:
                        abstr = ch(ch(bookNode,'Abstract'),'AbstractText').firstChild.data
                    except AttributeError:
                        abstr = ch(ch(bookNode,'Abstract'),'AbstractText').toprettyxml("  ")
                        abstr = re.sub(r"\s+", " ", re.sub("<[^>]*>", "", abstr))
            else:
                raise Exception('Unrecognized record type, for id %d (keys: %s)' % (Ids[start+i], str(p.keys())) )
            
            Abstracts.append( (int(pmid), str(abstr)) )
            i += 1
    
    # Save Abstracts, as a dictionary indexed by Ids
    Abstracts = dict( Abstracts )
    
    pickle.dump( Abstracts, bz2.BZ2File( Abstracts_file, 'wb' ) )

Taking a look at one paper's abstract:

In [26]:
Abstracts[30248106]

'Influenza A virus (IAV) in swine is a pathogen that causes a threat to the health as well as to the production of swine. Moreover, swine can spread this virus to other species including humans. The virus persists in different types of swine farms as evident in a number of studies. The core objectives of this study are (i) to analyze the dynamics of influenza infection of a farrow-to-finish swine farm, (ii) to explore the reinfection at the farm level, and finally (iii) to examine the effectiveness of two control strategies: vaccination and reduction of indirect contact. The analyses are conducted using a deterministic Susceptible-Exposed-Infectious-Recovered (SEIR) model. Simulation results show that the disease is maintained in gilts and piglets because of new susceptible pigs entering the population on a weekly basis. A sensitivity analysis shows that the results are not sensitive to variation in the parameters. The results of the reinfection simulation indicate that the virus persi

## ELink: Searching for related items in NCBI Entrez

To understand how to obtain paper citations with Entrez, we will first assemble a small set of PubMed IDs, and then query for their citations.
To that end, we search here for papers published in the Nature journal with our given keyword.

In [27]:
CA_search_term = search_term+'[TIAB] AND Nature[JOUR]'
CA_ids = Entrez.read( Entrez.esearch( db="pubmed", term=CA_search_term ) )['IdList']
CA_ids

['30232458', '30089927', '29925952', '29769722', '29618816', '29144466', '28953867', '28002402', '27749818', '26738596', '26503046', '26416728', '26123020', '26053121', '25762140', '25533958', '25519129', '25409151', '25409142', '25383540']

In [28]:
CA_summ = {
    p['Id'] : ( p['Title'], p['AuthorList'], p['PubDate'][:4], p['FullJournalName'], p.get('DOI', '') )
    for p in Entrez.read( Entrez.esummary(db="pubmed", id=','.join( CA_ids )) )
    }
CA_summ

{'25383540': ('p63(+)Krt5(+) distal airway stem cells are essential for lung regeneration.',
  ['Zuo W', 'Zhang T', 'Wu DZ', 'Guan SP', 'Liew AA', 'Yamamoto Y', 'Wang X', 'Lim SJ', 'Vincent M', 'Lessard M', 'Crum CP', 'Xian W', 'McKeon F'],
  '2015',
  'Nature',
  '10.1038/nature13903'),
 '25409142': ('Structure of influenza A polymerase bound to the viral RNA promoter.',
  ['Pflug A', 'Guilligay D', 'Reich S', 'Cusack S'],
  '2014',
  'Nature',
  '10.1038/nature14008'),
 '25409151': ('Structural insight into cap-snatching and RNA synthesis by influenza polymerase.',
  ['Reich S', 'Guilligay D', 'Pflug A', 'Malet H', 'Berger I', 'Crépin T', 'Hart D', 'Lunardi T', 'Nanao M', 'Ruigrok RW', 'Cusack S'],
  '2014',
  'Nature',
  '10.1038/nature14009'),
 '25519129': ('Influenza: An RNA-synthesizing machine.',
  ['Krug RM'],
  '2014',
  'Nature',
  '10.1038/516338a'),
 '25533958': ('Lineage-negative progenitors mobilize to regenerate lung epithelium after major injury.',
  ['Vaughan AE', 'Bru

Because we restricted our search to papers in an open-access journal, you can then follow their DOIs to freely access their PDFs at the journal's website.

We will now issue calls to `Entrez.elink()` using these PubMed IDs, to retrieve the IDs of papers that cite them.
The database from which the IDs will be retrieved is [PubMed Central](http://www.ncbi.nlm.nih.gov/pmc/), a free digital database of full-text scientific literature in the biomedical and life sciences.

A complete list of the kinds of links you can retrieve with `Entrez.elink()` can be found [here](http://eutils.ncbi.nlm.nih.gov/entrez/query/static/entrezlinks.html).

In [29]:
CA_citing = {
    id : Entrez.read( Entrez.elink(
            cmd = "neighbor",               # ELink command mode: "neighbor", returns
                                            #     a set of UIDs in `db` linked to the input UIDs in `dbfrom`.
            dbfrom = "pubmed",              # Database containing the input UIDs: PubMed
            db = "pmc",                     # Database from which to retrieve UIDs: PubMed Central
            LinkName = "pubmed_pmc_refs",   # Name of the Entrez link to retrieve: "pubmed_pmc_refs", gets
                                            #     "Full-text articles in the PubMed Central Database that cite the current articles"
            from_uid = id                   # input UIDs
            ) )
    for id in CA_ids
    }

In [30]:
CA_citing['28953867']

[{'ERROR': [], 'LinkSetDb': [{'Link': [{'Id': '6147258'}, {'Id': '6127164'}, {'Id': '6126728'}, {'Id': '6078801'}, {'Id': '6003312'}, {'Id': '6002944'}, {'Id': '6001749'}, {'Id': '5980710'}, {'Id': '5930012'}, {'Id': '5911685'}, {'Id': '5899735'}, {'Id': '5899712'}], 'DbTo': 'pmc', 'LinkName': 'pubmed_pmc_refs'}], 'LinkSetDbHistory': [], 'DbFrom': 'pubmed', 'IdList': ['28953867']}]

We have in `CA_citing[paper_id][0]['LinkSetDb'][0]['Link']` the list of papers citing `paper_id`. To get it as just a list of ids, we can do

In [31]:
cits = [ l['Id'] for l in CA_citing['28953867'][0]['LinkSetDb'][0]['Link'] ]
cits

['6147258',
 '6127164',
 '6126728',
 '6078801',
 '6003312',
 '6002944',
 '6001749',
 '5980710',
 '5930012',
 '5911685',
 '5899735',
 '5899712']

However, one more step is needed, as what we have now are PubMed Central IDs, and not PubMed IDs. Their conversion can be achieved through an additional call to `Entrez.elink()`:

In [32]:
cits_pm = Entrez.read( Entrez.elink( dbfrom="pmc", db="pubmed", LinkName="pmc_pubmed", from_uid=",".join(cits)) )
cits_pm

[{'ERROR': [], 'LinkSetDb': [{'Link': [{'Id': '30158186'}, {'Id': '29695210'}, {'Id': '29396418'}, {'Id': '29389072'}, {'Id': '29243094'}, {'Id': '29214605'}], 'DbTo': 'pubmed', 'LinkName': 'pmc_pubmed'}], 'LinkSetDbHistory': [], 'DbFrom': 'pmc', 'IdList': ['6147258', '6127164', '6126728', '6078801', '6003312', '6002944', '6001749', '5980710', '5930012', '5911685', '5899735', '5899712']}]

In [33]:
ids_map = { pmc_id : link['Id'] for (pmc_id,link) in zip(cits_pm[0]['IdList'], cits_pm[0]['LinkSetDb'][0]['Link']) }
ids_map

{'6002944': '29214605',
 '6003312': '29243094',
 '6078801': '29389072',
 '6126728': '29396418',
 '6127164': '29695210',
 '6147258': '30158186'}

And to check these papers:

In [34]:
{   p['Id'] : ( p['Title'], p['AuthorList'], p['PubDate'][:4], p['FullJournalName'], p.get('DOI', '') )
    for p in Entrez.read( Entrez.esummary(db="pubmed", id=','.join( ids_map.values() )) )
    }

{'29214605': ('Creation of artificial protein-protein interactions using α-helices as interfaces.',
  ['Yagi S', 'Akanuma S', 'Yamagishi A'],
  '2018',
  'Biophysical reviews',
  '10.1007/s12551-017-0352-9'),
 '29243094': ('Hierarchical design of artificial proteins and complexes toward synthetic structural biology.',
  ['Arai R'],
  '2018',
  'Biophysical reviews',
  '10.1007/s12551-017-0376-1'),
 '29389072': ('Structures and disulfide cross-linking of de novo designed therapeutic mini-proteins.',
  ['Silva DA', 'Stewart L', 'Lam KH', 'Jin R', 'Baker D'],
  '2018',
  'The FEBS journal',
  '10.1111/febs.14394'),
 '29396418': ('Structural insights into the design of novel anti-influenza therapies.',
  ['Wu NC', 'Wilson IA'],
  '2018',
  'Nature structural & molecular biology',
  '10.1038/s41594-018-0025-9'),
 '29695210': ('Recent advances in automated protein design and its future challenges.',
  ['Setiawan D', 'Brender J', 'Zhang Y'],
  '2018',
  'Expert opinion on drug discovery',
  '

### Citations dataset

We have now seen all the steps required to assemble a dataset of citations to each of the papers in our dataset.

In [35]:
Citations_file = 'data/' + search_term + '_Citations.pkl.bz2'
Citations = []

At least one server query will be issued per paper in `Ids`. Because NCBI allows for at most 3 queries per second (see [here](http://biopython.org/DIST/docs/api/Bio.Entrez-pysrc.html#_open)), this dataset will take a long time to assemble. Should you need to interrupt it for some reason, or the connection fail at some point, it is safe to just rerun the cell below until all data is collected.

In [38]:
import http.client

if Citations == [] and os.path.exists( Citations_file ):
    Citations = pickle.load( bz2.BZ2File( Citations_file, 'rb' ) )

if len(Citations) < len(Ids):
    
    i = len(Citations)
    checkpoint = int(len(Ids) / 100) + 1      # save to hard drive at every 1% of Ids fetched
    
    for pm_id in Ids[i:]:               # either starts from index 0, or resumes from where we previously left off
        
        while True:
            try:
                # query for papers archived in PubMed Central that cite the paper with PubMed ID `pm_id`
                c = Entrez.read( Entrez.elink( dbfrom = "pubmed", db="pmc", LinkName = "pubmed_pmc_refs", id=str(pm_id) ) )
                
                c = c[0]['LinkSetDb']
                if len(c) == 0:
                    # no citations found for the current paper
                    c = []
                else:
                    c = [ l['Id'] for l in c[0]['Link'] ]
                    
                    # convert citations from PubMed Central IDs to PubMed IDs
                    p = []
                    retrieve_per_query = 500
                    for start in range( 0, len(c), retrieve_per_query ):
                        query_ids = ','.join( c[start : start+retrieve_per_query] )
                        r = Entrez.read( Entrez.elink( dbfrom="pmc", db="pubmed", LinkName="pmc_pubmed", from_uid=query_ids ) )
                        # select the IDs. If no matching PubMed ID was found, [] is returned instead
                        p.extend( [] if r[0]['LinkSetDb']==[] else [ int(link['Id']) for link in r[0]['LinkSetDb'][0]['Link'] ] )
                    c = p
            
            except http.client.BadStatusLine:
                # Presumably, the server closed the connection before sending a valid response. Retry until we have the data.
                print('r')
                continue
            except HTTPError:
                print('r')
                continue
            break
        
        Citations.append( (pm_id, c) )
        if (i % 10000 == 0):
            print('')
            print(i, end='')
        if (i % 100 == 0):
            print('.', end='')
        i += 1
        
        if i % checkpoint == 0:
            print('\tsaving at checkpoint', i)
            pickle.dump( Citations, bz2.BZ2File( Citations_file, 'wb' ) )
    
    print('\n done.')
    
    # Save Citations, as a dictionary indexed by Ids
    Citations = dict( Citations )
    
    pickle.dump( Citations, bz2.BZ2File( Citations_file, 'wb' ) )

.......	saving at checkpoint 75422
........	saving at checkpoint 76299
.........	saving at checkpoint 77176
.........	saving at checkpoint 78053
.........	saving at checkpoint 78930
.........	saving at checkpoint 79807
.
80000.......	saving at checkpoint 80684
.........	saving at checkpoint 81561
.........	saving at checkpoint 82438
.........	saving at checkpoint 83315
........	saving at checkpoint 84192
.........	saving at checkpoint 85069
.........	saving at checkpoint 85946
.........	saving at checkpoint 86823
........
 done.


To see that we have indeed obtained the data we expected, you can match the ids below, with the ids listed at the end of last section.

In [39]:
Citations[28953867]

[30158186, 29695210, 29396418, 29389072, 29243094, 29214605]

## Where do we go from here?

Running the code above generates multiple local files, containing the datasets we'll be working with. Loading them into memory is a matter of just issuing a call like<br>
``data = pickle.load( bz2.BZ2File( data_file, 'rb' ) )``.

The Entrez module will therefore no longer be needed, unless you wish to extend your data processing with additional information retrieved from PubMed.

Should you be interested in looking at alternative ways to handle the data, have a look at the [sqlite3](http://docs.python.org/3/library/sqlite3.html) module included in Python's standard library, or [Pandas](http://pandas.pydata.org/), the Python Data Analysis Library.